In [28]:
from pathlib import Path
from skimage import io
import tifffile
import os
import napari
import pyclesperanto_prototype as cle
from skimage.measure import regionprops_table
import pandas as pd

In [29]:
def read_images(directory_path):
    """Reads all the images in the input path and organizes them according to the well_id"""
    # Define the directory containing your files
    directory_path = Path(directory_path)

    # Initialize a dictionary to store the grouped (per position) files
    images_per_position = {}

    # Iterate through the files in the directory
    for file_path in directory_path.glob("*"):
        # Check if the path is a file and ends with ".tif"
        if file_path.is_file() and file_path.suffix.lower() == ".tif":
            # Get the filename without the extension
            filename = file_path.stem
            # Remove unwanted files (Plate_R files)
            if "ch02" in filename:
                pass
            # Remove any other unwanted files
            elif "_z" not in filename:
                pass
            else:
                # Extract the last part of the filename (e.g., 1_Crop001_z00_ch00)
                last_part = filename.split(" ")[1]

                # Get the first three letters to create the group name (position_id)
                position_id = last_part[:1]

                # Check if the well_id exists in the dictionary, if not, create a new list
                if position_id not in images_per_position:
                    images_per_position[position_id] = []

                # Append the file to the corresponding group
                images_per_position[position_id].append(str(file_path))

    return images_per_position

def create_stack(image_paths):
    """Takes a collection of image paths containing individual z-stacks and returns a stack of images"""
    # Load images from the specified paths
    image_collection = io.ImageCollection(image_paths)
    # Stack images into a single 3D numpy array
    stack = io.concatenate_images(image_collection)
    
    return stack

def save_stacks(images_per_position, output_dir="./output/processed_stacks"):
    """Takes a images_per_position from read_images as input, stacks them on a per channel basis and saves the resulting images on a per position basis"""
    for position_id, files in images_per_position.items():
        
        ch00_paths = []
        ch01_paths = []
        
        for image_path in images_per_position[position_id]:
            if "ch00" in image_path:
                ch00_paths.append(image_path)
            elif "ch01" in image_path:
                ch01_paths.append(image_path)
                
        
        # Generate the stacks
        ch00_stack = create_stack(ch00_paths)
        ch01_stack = create_stack(ch01_paths)

        # Create a directory to store the tif files if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Construct the output file path
        output_path_ch00 = os.path.join(output_dir, f"Position {position_id}_ch00.tif")
        output_path_ch01 = os.path.join(output_dir, f"Position {position_id}_ch01.tif")

        # Save the resulting minimum projection
        tifffile.imwrite(output_path_ch00, ch00_stack)
        tifffile.imwrite(output_path_ch01, ch01_stack)
        
def return_stacks(images_per_position):
    """Takes a images_per_position from read_images as input, stacks them on a per channel basis and returns the stacks"""
    for position_id, files in images_per_position.items():
        
        ch00_paths = []
        ch01_paths = []
        
        for image_path in images_per_position[position_id]:
            if "ch00" in image_path:
                ch00_paths.append(image_path)
            elif "ch01" in image_path:
                ch01_paths.append(image_path)
                
        
        # Generate the stacks
        ch00_stack = create_stack(ch00_paths)
        ch01_stack = create_stack(ch01_paths)

        return ch00_stack, ch01_stack

In [30]:
images_per_position = read_images("./raw_data/ANP32A_N4/Crop")

In [31]:
images_per_position

{'1': ['raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z00_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z00_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z01_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z01_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z02_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z02_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z03_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z03_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z04_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z04_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z05_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z05_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z06_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z06_ch01.tif',
  'raw_data\\ANP32A_N4\\Crop\\Position 1_Crop001_z07_ch00.tif',
  'raw_data\\ANP32A_N4\\Crop\\Posit

In [32]:
position_id = str(1)

ch00_paths = []
ch01_paths = []

for image_path in images_per_position[position_id]:
    if "ch00" in image_path:
        ch00_paths.append(image_path)
    elif "ch01" in image_path:
        ch01_paths.append(image_path)
        

# Generate the stacks
ch00_stack = create_stack(ch00_paths)
ch01_stack = create_stack(ch01_paths)

In [34]:
viewer = napari.Viewer(ndisplay=2)

scaling_x_um = 0.342
scaling_y_um = 0.342
scaling_z_um = 0.663

multiplier = 1 / scaling_x_um

scaling_x_um = scaling_x_um * multiplier
scaling_y_um = scaling_y_um * multiplier
scaling_z_um = scaling_z_um * multiplier

nuclei_resampled = cle.scale(ch00_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
marker_resampled = cle.scale(ch01_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)
segmented = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=10, outline_sigma=1)

viewer.add_image(nuclei_resampled)
viewer.add_image(marker_resampled)
viewer.add_labels(segmented)


<Labels layer 'segmented' at 0x23403d0a820>

In [35]:
eroded_labels = cle.erode_labels(segmented, radius=2)

In [36]:
viewer.add_labels(eroded_labels)

<Labels layer 'eroded_labels' at 0x23402a44a30>

In [37]:
eroded_labels_np = cle.pull(eroded_labels)
marker_resampled_np = cle.pull(marker_resampled)

# Extract regionprops
props = regionprops_table(label_image=eroded_labels_np, intensity_image=marker_resampled_np, properties=["label", "intensity_mean", "intensity_max", "centroid", "area_filled"])

# Construct a dataframe
df = pd.DataFrame(props)

df

,label,intensity_mean,intensity_max,centroid-0,centroid-1,centroid-2,area_filled
0,1,3.238003,94.0,25.293695,423.522397,13.547618,17581.0
1,2,8.113064,150.0,19.712036,479.831151,14.510693,12064.0
2,3,12.092316,255.0,20.532735,374.298589,25.086563,14602.0
3,4,4.493991,122.0,18.837699,88.217976,34.919979,9735.0
4,5,8.865438,255.0,21.138917,542.655738,49.078125,11712.0
...,...,...,...,...,...,...,...
78,79,5.249406,216.0,16.449789,54.277118,701.981196,15181.0
79,80,5.073774,255.0,20.679506,127.201149,707.535092,15317.0
80,81,4.845489,84.0,19.521525,8.919599,714.129349,5087.0
81,82,4.045872,18.0,14.146789,321.302752,718.899083,109.0


In [38]:
intensity_mean = df["intensity_mean"].mean()
intensity_mean

8.541831051010684